In [ ]:
# run against production 2025-06-03
from tqdm import tqdm
from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist
from django.db import IntegrityError
from apis_core.utils import get_object_from_pk_or_uri
from AcdhArcheAssets.uri_norm_rules import get_normalized_uri
import pandas as pd

In [ ]:
col, _ = Collection.objects.get_or_create(name="Schnitzler-Kino")

In [ ]:
df = gsheet_to_df("1PjIbMdRQAA1erSfn88EDTTTtIyx0zLqYiWKp1FGFFis")

In [ ]:
for g, ndf in df.groupby("target_id"):
    row = ndf.iloc[0]
    item = TempEntityClass.objects.get(id=g)
    if not "nodate" in row.target_start_date_written:
        item.start_date_written = row.target_start_date_written
        item.save()

In [ ]:
for g, ndf in df.groupby("source_id"):
    row = ndf.iloc[0]
    item = Person.objects.get(id=g)
    if not "nodate" in row.source_start_date_written:
        item.start_date_written = row.source_start_date_written
        item.save()

In [ ]:
df.replace("nodate", "", inplace=True)

In [ ]:
does_not_exist = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        item = PersonWork.objects.get(id=row["relation_pk"])
    except ObjectDoesNotExist:
        does_not_exist.append(row["relation_pk"])
    relation_type = PersonWorkRelation.objects.get(name=row["relation_type"])
    item.relation_type = relation_type
    item.related_person = Person.objects.get(id=row["source_id"])
    item.related_work = Work.objects.get(id=row["target_id"])
    item.start_date_written = row["relation_start_date_written"]
    item.end_date_written = row["relation_end_date_written"]
    item.save()
    

In [ ]:
for x in does_not_exist:
    print(x)